# Les intellectuels

## Méthodologie :
En parcourant la base de données DBpedia, construite sur la base des informations extraites de Wikipédia, je suis tombé sur une classe concernant les intellectuels : https://dbpedia.org/class/yago/Intellectual109621545.

À partir du point d’accès SPARQL (https://dbpedia.org/sparql) j’ai récupéré les données de cette classe. Elle contient 197'176 personnes, qui sont donc considérées comme intellectuelles selon DBpedia. Ensuite, j’ai créé une base données locales afin de stockées ces données.

## Importation des librairies

In [1]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import pprint
import csv
import ast
import re

from collections import Counter
from operator import itemgetter

import sqlite3 as sql
import time

from importlib import reload
from shutil import copyfile

In [3]:
from ipywidgets import (
    interact, IntSlider, Checkbox,
    interactive, SelectMultiple)

In [4]:
import sparql_functions as spqf

In [5]:
reload(spqf)

<module 'sparql_functions' from 'C:\\jupyter_notebooks\\intellectuels_JS\\sparql_functions.py'>

In [6]:
db = 'data/intellectuels.db'

## Inspection des propriétés disponibles

Nous sortons ici la liste des propriétés les mieux renseignées à propos de la liste des intellectuels.

In [44]:
query_intellectuals_properties = """
PREFIX  yago: <http://dbpedia.org/class/yago/>
PREFIX  foaf: <http://xmlns.com/foaf/0.1/>

SELECT DISTINCT ?p ?direction (count(*) AS ?freq)
WHERE
  {  { ?person  a                     yago:Intellectual109621545 ;
                     ?p                    ?o
            BIND("out" AS ?direction)
          }
     
    UNION
          { ?person  a                     yago:Intellectual109621545 .
            ?s       ?p                    ?person
            BIND("in" AS ?direction)
          }      
  }
GROUP BY ?p ?direction
ORDER BY DESC(?freq)

"""


# Exécution de la requête qui retournera un dictionnaire JSON
sparql = SPARQLWrapper("https://dbpedia.org/sparql")
sparql.setQuery(query_intellectuals_properties)
sparql.setReturnFormat(JSON)
rc = sparql.queryAndConvert()


# Transformation des résultats en liste puis en dataframe comprenant les propriétés et leurs fréquences
l_prop = spqf.sparql_result_to_list_vars(rc, '?p ?freq')
df_prop = pd.DataFrame(l_prop, columns=['Property','Frequency'])

# Affichage des 50 propriétés les mieux renseignées
df_prop[:50]

,Property,Frequency
0,http://dbpedia.org/ontology/wikiPageWikiLink,9405330
1,http://dbpedia.org/ontology/wikiPageWikiLink,8047534
2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,6697687
3,http://www.w3.org/2002/07/owl#sameAs,2618194
4,http://purl.org/dc/terms/subject,2129616
5,http://dbpedia.org/property/wikiPageUsesTemplate,1658868
6,http://www.w3.org/2000/01/rdf-schema#label,862286
7,http://www.w3.org/2000/01/rdf-schema#comment,850919
8,http://dbpedia.org/ontology/abstract,850919
9,http://dbpedia.org/ontology/wikiPageExternalLink,508640


Au vu des données disponibles et de leur pertinence, nous avons fait le choix de retenir les propriétés suivantes afin de procéder à une analyse de données :

1.	Birth year
2.	Birth place
3.	Occupation
4.	Academic discipline
5.	Abstract 

Remarque : 
Au vu de la thématique du cours, il aurait été intéressant de pouvoir compter la proportion d’hommes et de femmes sur les 197'176 personnes. Malheureusement, la propriété « gender » n’est pas renseignée.


## Analyses de données

Question de départ : quelles sont les tendances structurantes de la population des intellectuels recensée sur DBpedia ?

L’intérêt principal de l’analyse de données est la création d’un modèle sur la base des données extraites de Wikipédia. Ainsi, nous pourrons donner des éléments de réponse aux questions suivantes :
-	Comment les naissances des intellectuels sont-elles distribuées dans le temps ?
-	Comment les naissances des intellectuels sont-elles distribuées dans l’espace ?
-	Quelles sont les « occupations » principales de intellectuels ?
-	Quelles sont les disciplines académiques principales des intellectuels ?
-	Quels sont les intellectuels qui ont un lien avec le féminisme ?



### Années de naissance

Lien du carnet : https://github.com/jorisstacheli/intellectuels_JS/blob/main/1_annees_naissances_intellectuels.ipynb

### Lieux de naissance

Lien du carnet : https://github.com/jorisstacheli/intellectuels_JS/blob/main/2_lieux_naissances_intellectuels.ipynb

### Occupations

Lien du carnet : https://github.com/jorisstacheli/intellectuels_JS/blob/main/3_occupations_intellectuels.ipynb

### Discplines académiques

Lien du carnet : https://github.com/jorisstacheli/intellectuels_JS/blob/main/4_disciplines_academiques_intellectuels.ipynb

### Résumés

Lien du carnet : https://github.com/jorisstacheli/intellectuels_JS/blob/main/5_resume_intellectuels.ipynb

## Limites

-	Il ne s’agit que d’une ébauche, mes capacités en analyse de données sont limitées.
-	Nous sommes soumis au jugement de Wikipédia quant à la liste des personnes considérées comme "intellectuelles"
-	Il peut y avoir des erreurs dans les données (nous verrons un exemple dans le carnet 1)
-	Certaines données sont peu renseignées et les conclusions doivent être mesurées (cf. carnets 3 et 4)


